### first 그대로 하고, KFold만 바꿔 보기
- 결과: 0.80319
- 단순 iter할 때와 비교(0.80210): 좋아짐

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style='ticks') # 그래프를 seaborn 스타일로 그리기
plt.rc('font', family = 'Malgun Gothic') # 한글 처리

from sklearn.linear_model import LogisticRegression
import scipy
from tqdm import tqdm_notebook as tqdm

In [2]:
%%time
train = pd.read_csv("../input/cat-in-the-dat/train.csv")
test = pd.read_csv("../input/cat-in-the-dat/test.csv")

print(train.shape)
print(test.shape)

(300000, 25)
(200000, 24)
CPU times: user 2.22 s, sys: 250 ms, total: 2.47 s
Wall time: 2.34 s


In [3]:
ddall = train.append(test, sort = False)
num_train = len(train)
print(test.shape[0])
print(ddall['target'].isnull().sum())
ddall.head()

200000
200000


,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target
0,0,0,0,0,T,Y,Green,Triangle,Snake,Finland,...,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2,0.0
1,1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,...,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8,0.0
2,2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,...,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2,0.0
3,3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,...,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1,1.0
4,4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,...,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8,0.0


In [4]:
drop_cols = ['bin_0']
#drop_cols = []
ddall['ord_5a'] = ddall['ord_5'].str[0]
ddall['ord_5b'] = ddall['ord_5'].str[1]
drop_cols.append('ord_5')

In [5]:
# train, test 모두에 들어있지 않는 값을 single value('xor')로 변경
for col in ['nom_5', 'nom_6', 'nom_7', 'nom_8', 'nom_9']:
    train_vals = set(train[col].unique())
    test_vals = set(test[col].unique())
    
    # ^ 연산 -> XOR(같으면 0, 다르면 1). 교집합 제외한 나머지가 남음
    xor_cat_vals = train_vals ^ test_vals
    print("XOR 값 수:", len(xor_cat_vals))
    if xor_cat_vals: # 값이 있으면 loop 안으로 들어감. 참고해 두기!!
        ddall.loc[ddall[col].isin(xor_cat_vals), col] = 'xor'

XOR 값 수: 0
XOR 값 수: 0
XOR 값 수: 1
XOR 값 수: 9
XOR 값 수: 316


In [6]:
%%time
# 전부 One-Hot Encoding
X = ddall[ddall.columns.difference(['id', 'target'] + drop_cols)] #해당 칼럼들을 뺀 것.
# 아래 6개 칼럼은 원핫 안함. 왜 이것들만??
X_oh = X[X.columns.difference(['ord_1', 'ord_4', 'ord_5a', 'ord_5b', 'day', 'month'])]
oh1 = pd.get_dummies(X_oh, columns = X_oh.columns, drop_first=True, sparse=True)
ohc1 = oh1.to_coo()
ohc1

CPU times: user 16.9 s, sys: 560 ms, total: 17.4 s
Wall time: 14.2 s


<500000x15968 sparse matrix of type '<class 'numpy.uint8'>'
	with 6430440 stored elements in COOrdinate format>

In [7]:
from sklearn.base import TransformerMixin
from itertools import repeat

In [8]:
class ThermometerEncoder(TransformerMixin):
    def __init__(self, sort_key=None):
        self.sort_key = sort_key
        self.value_map_ = None
    
    def fit(self, X, y=None):
        self.value_map_ = {val: i for i, val in enumerate(sorted(X.unique(), key=self.sort_key))}
        return self
    
    def transform(self, X, y=None):
        values = X.map(self.value_map_)
        
        possible_values = sorted(self.value_map_.values())
        
        idx1 = []
        idx2 = []
        
        all_indices = np.arange(len(X))
        
        for idx, val in enumerate(possible_values[:-1]):
            new_idxs = all_indices[values > val]
            idx1.extend(new_idxs)
            idx2.extend(repeat(idx, len(new_idxs)))
            
        result = scipy.sparse.coo_matrix(
            ([1] * len(idx1), (idx1, idx2)), 
            shape=(len(X), len(possible_values)), dtype="int8")
            
        return result

In [9]:
%%time
thermos = []
for col in ["ord_1", "ord_2", "ord_3", "ord_4", "ord_5a", "day", "month"]:
    if col == 'ord_1':
        sort_key = ['Novice', 'Contributor', 'Expert', 'Master', 'Grandmaster'].index
    elif col == 'ord_2':
        sort_key = ['Freezing', 'Cold', 'Warm', 'Hot', 'Boiling Hot', 'Lava Hot'].index
    elif col in ['ord_3', 'ord_4', 'ord_5a']: #알파벳 데이터. 왜 ord_5b는 안하지??
        sort_key = str
    elif col in ['day', 'month']:
        sort_key = int
    else:
        raise ValueError(col)
        
    encoder = ThermometerEncoder(sort_key=sort_key)
    thermos.append(encoder.fit_transform(X[col]))

CPU times: user 13 s, sys: 1.15 s, total: 14.2 s
Wall time: 14.5 s


In [10]:
ohc = scipy.sparse.hstack([ohc1] + thermos).tocsr()
display(ohc)

<500000x16091 sparse matrix of type '<class 'numpy.int16'>'
	with 34244885 stored elements in Compressed Sparse Row format>

In [11]:
X_train = ohc[:num_train]
X_test = ohc[num_train:]
y_train = train['target'].values

print(X_train.shape) #아까 X 걸러내는 과정에서 target 칼럼 뺐음 -> test와 차원이 같음
print(X_test.shape)

(300000, 16091)
(200000, 16091)


In [12]:
%%time
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score as auc
from sklearn.linear_model import LogisticRegression

# Model
def run_cv_model(train, test, target, model_fn, params={}, eval_fn=None, label='model'):
    kf = KFold(n_splits=5)
    fold_splits = kf.split(train, target)
    cv_scores = []
    pred_full_test = 0
    pred_train = np.zeros((train.shape[0]))
    i = 1
    for dev_index, val_index in fold_splits:
        print('Started ' + label + ' fold ' + str(i) + '/5')
        dev_X, val_X = train[dev_index], train[val_index]
        dev_y, val_y = target[dev_index], target[val_index]
        params2 = params.copy()
        pred_val_y, pred_test_y = model_fn(dev_X, dev_y, val_X, val_y, test, params2)
        pred_full_test = pred_full_test + pred_test_y
        pred_train[val_index] = pred_val_y
        if eval_fn is not None:
            cv_score = eval_fn(val_y, pred_val_y)
            cv_scores.append(cv_score)
            print(label + ' cv score {}: {}'.format(i, cv_score))
        i += 1
    print('{} cv scores : {}'.format(label, cv_scores))
    print('{} cv mean score : {}'.format(label, np.mean(cv_scores)))
    print('{} cv std score : {}'.format(label, np.std(cv_scores)))
    pred_full_test = pred_full_test / 5.0
    results = {'label': label,
              'train': pred_train, 'test': pred_full_test,
              'cv': cv_scores}
    return results


def runLR(train_X, train_y, test_X, test_y, test_X2, params):
    print('Train LR')
    model = LogisticRegression(**params)
    model.fit(train_X, train_y)
    print('Predict 1/2')
    pred_test_y = model.predict_proba(test_X)[:, 1]
    print('Predict 2/2')
    pred_test_y2 = model.predict_proba(test_X2)[:, 1]
    return pred_test_y, pred_test_y2

lr_params = {'solver': 'lbfgs', 'C': 0.1}
results = run_cv_model(X_train, X_test, y_train, runLR, lr_params, auc, 'lr')

Started lr fold 1/5
Train LR


/Users/qpdev/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Predict 1/2
Predict 2/2
lr cv score 1: 0.7959661308502421
Started lr fold 2/5
Train LR


/Users/qpdev/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Predict 1/2
Predict 2/2
lr cv score 2: 0.7962328656512232
Started lr fold 3/5
Train LR


/Users/qpdev/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Predict 1/2
Predict 2/2
lr cv score 3: 0.8033901113752656
Started lr fold 4/5
Train LR


/Users/qpdev/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Predict 1/2
Predict 2/2
lr cv score 4: 0.7986581335298614
Started lr fold 5/5
Train LR
Predict 1/2
Predict 2/2
lr cv score 5: 0.7999214453533618
lr cv scores : [0.7959661308502421, 0.7962328656512232, 0.8033901113752656, 0.7986581335298614, 0.7999214453533618]
lr cv mean score : 0.7988337373519908
lr cv std score : 0.002718903833948214
CPU times: user 1min 8s, sys: 1.99 s, total: 1min 10s
Wall time: 36.6 s


/Users/qpdev/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


In [13]:
%%time
submission = pd.DataFrame({'id': test['id'], 'target': results['test']})
submission.to_csv("../output/cat_chal/1104_submission_kfold_comparison.csv", index = False)

CPU times: user 840 ms, sys: 40.5 ms, total: 880 ms
Wall time: 923 ms
